# Playbook Explorer

This notebook is intended to be a live example of how to work with SysML v2 models at analysis-time. For these purposes, the following terms are introduced:
* An *interpretation* is the mapping of user model symbols (the "M1 model" in OMG-speak) into semantically-correct symbols that represent real world objects meant to conform to the model (the "M0" in OMG-speak). Interpretation semantics are inspired by https://www.w3.org/TR/owl2-direct-semantics/ and are mostly similar.
* A *sequence* for an interpretation contains *atoms* or *instances* that match to real world things. Reading a sequence from left to right provides a set of nested contexts for the atoms that is important to the interpretation. For example [Rocket#0, LS#3] is a 2-sequence to describe facts around the LS#3 atom when it is considered in context for Rocket#0. This is an important idea for the SysML time and occurrence model where one may want to see how values change under different conditions.

This is a notebook that walks through the random interpretation generator to help developers working on their own interpreters.

## Example Model

The model that is used for this example is the SysML v2 Validation case 2a:

    package '2a-Parts Interconnection' {
	import Definitions::*;
	import Usages::*;

	package Definitions {		
		// Port Definitions
		
		port def FuelCmdPort;
		
		port def DrivePwrPort;
		port def ClutchPort;
		
		port def ShaftPort_a;
		port def ShaftPort_b;
		port def ShaftPort_c;
		port def ShaftPort_d;
		
		port def DiffPort;
		port def AxlePort;
		port def AxleToWheelPort;
		port def WheelToAxlePort;
		port def WheelToRoadPort;
		
		/**
		 * A port definition can have nested ports.
		 */
		port def VehicleToRoadPort { 
			port wheelToRoadPort: WheelToRoadPort[2];
		}
	
		// Blocks
	
		part def VehicleA { 
			port fuelCmdPort: FuelCmdPort;
			port vehicleToRoadPort: VehicleToRoadPort;
		}
		
		part def AxleAssembly;		
		part def RearAxleAssembly :> AxleAssembly { 
			port shaftPort_d: ShaftPort_d;
		}
		
		part def Axle;
		part def RearAxle :> Axle;
		
		part def HalfAxle { 
			port axleToDiffPort: AxlePort;
			port axleToWheelPort: AxleToWheelPort;
		}
		
		part def Engine { 
			port fuelCmdPort: FuelCmdPort;
			port drivePwrPort: DrivePwrPort;
		}
	
		part def Transmission { 
			port clutchPort: ClutchPort;
			port shaftPort_a: ShaftPort_a;
		}
		
		part def Driveshaft { 
			port shaftPort_b: ShaftPort_b;
			port shaftPort_c: ShaftPort_c;
		}	
		
		/**
		 * Ports do not have to be defined on part defs.
		 * They can be added directly to their usages.
		 */
		part def Differential;
		part def Wheel;
		
		// Interface Definitions
		
		/**
		 * The ends of an interface definition are always ports.
		 */
		interface def EngineToTransmissionInterface {
			end drivePwrPort: DrivePwrPort;
			end clutchPort: ClutchPort;
		}
		
		interface def DriveshaftInterface {
			end shaftPort_a: ShaftPort_a;
			end shaftPort_d: ShaftPort_d;
			
			/**
			 * 'driveshaft' is a reference to the driveshaft that will
			 * act as the "interface medium" for this interface.
			 */
			ref driveshaft: Driveshaft {
				port shaftPort_b :>> Driveshaft::shaftPort_b;
				port shaftPort_c :>> Driveshaft::shaftPort_c;
			}
			
			/**
			 * The two ends of 'DriveShaftInterface' are always connected
			 * via the referenced 'driveshaft'.
			 */
			connect shaftPort_a to driveshaft::shaftPort_b;
			connect driveshaft::shaftPort_c to shaftPort_d;
		}
		
	}
	
	package Usages {
	
		part vehicle1_c1: VehicleA {
			
			/**
			 * Inherited ports are redefined to provide "local connection points".
			 * ("::>>" is a shorthand for "redefines".)
			 */
			port fuelCmdPort :>> VehicleA::fuelCmdPort;
			
			bind fuelCmdPort = engine::fuelCmdPort;
			
			part engine: Engine {
				port fuelCmdPort :>> Engine::fuelCmdPort;
				port drivePwrPort :>> Engine::drivePwrPort;
			}
			
			/**
			 * A usage of an interface definition connects two ports relative to 
			 * a containing context.
			 */
			interface :EngineToTransmissionInterface
				connect engine::drivePwrPort to transmission::clutchPort;
				
			part transmission: Transmission {
				port clutchPort :>> Transmission::clutchPort;
				port shaftPort_a :>> Transmission::shaftPort_a;	
			}
			
			/**
			 * This 'driveshaft' is the part of 'vehicle1_c1' that will act as the
			 * interface medium in the following 'DriveshaftInterface' usage.
			 */
			part driveshaft: Driveshaft;
			
			interface :DriveshaftInterface
				connect transmission::shaftPort_a to rearAxleAssembly::shaftPort_d {
					/**
					 * The reference property from 'DriveshaftInterface' is redefined
					 * in order to bind it to the appropriate part of 'vehicle1_c1'.
					 */
					ref :>> DriveshaftInterface::driveshaft = vehicle1_c1::driveshaft;
				}
	
			part rearAxleAssembly: RearAxleAssembly {
				port shaftPort_d: ShaftPort_d :>> RearAxleAssembly::shaftPort_d;
	
				bind shaftPort_d = differential::shaftPort_d;
				
				part differential: Differential {
					/**
					 * If the part def has no ports, then they can be defined directly in
					 * a usage of the part def.
					 */
					port shaftPort_d: ShaftPort_d;
					port leftDiffPort: DiffPort;
					port rightDiffPort: DiffPort;
				}
				
				/* TODO: Allow use of "interface" keyword for an unnamed, default-typed interface usage. */
				
				/**
				 * A connection can be to a port that is arbitrarily deeply nested, on either end. 
				 */
				connect differential::leftDiffPort to rearAxle::leftHalfAxle::leftAxleToDiffPort;
				connect differential::rightDiffPort to rearAxle::rightHalfAxle::rightAxleToDiffPort;
		
				part rearAxle: RearAxle {
					part leftHalfAxle: HalfAxle {
						port leftAxleToDiffPort :>> HalfAxle::axleToDiffPort;
						port leftAxleToWheelPort :>> HalfAxle::axleToWheelPort;
					}
					part rightHalfAxle: HalfAxle  {
						port rightAxleToDiffPort :>> HalfAxle::axleToDiffPort;
						port rightAxleToWheelPort :>> HalfAxle::axleToWheelPort;
					}
				}
				
				connect rearAxle::leftHalfAxle::leftAxleToWheelPort to leftWheel::wheelToAxlePort;
				connect rearAxle::rightHalfAxle::rightAxleToWheelPort to rightWheel::wheelToAxlePort;
	
				part rearWheel: Wheel[2] ordered;
				
				/* The two rear wheels of 'rearAxleAssembly' must be given
				 * their own names in order to be referenced in connections.
				 * 
				 * (":>" is a shorthand here for "subsets".)
				 */
				part leftWheel :> rearWheel = rearWheel[1] {
					port wheelToAxlePort: WheelToAxlePort;
					port wheelToRoadPort: WheelToRoadPort;
				}
				
				part rightWheel :> rearWheel = rearWheel[2] {
					port wheelToAxlePort: WheelToAxlePort;
					port wheelToRoadPort: WheelToRoadPort;
				}
				
			}
			
			bind rearAxleAssembly::leftWheel::wheelToRoadPort = 
				 vehicleToRoadPort::leftWheelToRoadPort;
				 
			bind rearAxleAssembly::rightWheel::wheelToRoadPort = 
				 vehicleToRoadPort::rightWheelToRoadPort;
				
			port vehicleToRoadPort redefines VehicleA::vehicleToRoadPort {
				port leftWheelToRoadPort :> wheelToRoadPort = wheelToRoadPort[1];
				port rightWheelToRoadPort :> wheelToRoadPort = wheelToRoadPort[2];
			}
			
		}
	
	}
	
}

## Imports

Import key modules, functions, and classes from the PyMBE library:

In [1]:
import pymbe.api as pm
from pymbe.label import get_label
import networkx as nx
import matplotlib.pyplot as plt

from pymbe.interpretation.interp_playbooks import *
from pymbe.interpretation.results import *

from pymbe.client import SysML2Client
from pymbe.query.metamodel_navigator import map_inputs_to_results

from pymbe.interpretation.calc_dependencies import generate_execution_order

import os

from pymbe.graph.lpg import SysML2LabeledPropertyGraph

## Key IDs

The unique identifiers below are useful references for walking through the interpretations generated in this notebook.

## Client Setup

The example here uses a local copy of the JSON file obtained by a GET operation on the SysML v2 API at:
http://sysml2-sst.intercax.com:9000/projects/a4f6a618-e4eb-4ac8-84b8-d6bcd3badcec/commits/c48aea9b-42fb-49b3-9a3e-9c39385408d7/elements?page[size]=5000

Create the client and load local data.

In [2]:
helper_client = SysML2Client()

path = "C:\\Users\\bjorn\\Documents\\Git\\pyMBE"
file_name = "\\tests\\data\\Validation Case 2a\\elements.json"

helper_client._load_disk_elements(path + str(file_name))

Create a graph representation of the model and load it into memory.

In [3]:
lpg = SysML2LabeledPropertyGraph()
lpg.update(helper_client.elements_by_id, False)

This is just a helper to make abbreviations more legible.

In [4]:
shorten_pre_bake = {
}

Create an interpretation of the Kerbal model using the random generator playbook. In general, this randomly selects:
- The ratios of partitioning abstract classifier sequence sets into concrete sets. For example, one draw may choose 2 liquid stages and 3 solids.
- The number of sequences to create for a given feature multiplicity. For example, draw 2 for a 0..8 engines : Liquid Engine PartUsage.

The playbook also attempts to make sequences created obey the Subsetting relationship (elements marked with subsets in M1 model should have their interpretation sequences entirely included within the interpretation sequences of the superset).

In [5]:
from pymbe.query.query import get_types_for_feature, get_features_typed_by_type, roll_up_multiplicity_for_type

In [6]:
get_types_for_feature(lpg, 'a0aba22e-a4ea-44b5-adde-928f9e6e30dd')

['afc897ef-d6ba-4a15-a6be-b947f3e1016f']

In [7]:
get_types_for_feature(lpg, '6656cd12-8549-4690-9e17-b858278507b3')

['165f4cd9-9849-45bf-b31b-d0b675df6774']

In [8]:
get_features_typed_by_type(lpg, '165f4cd9-9849-45bf-b31b-d0b675df6774')

['6656cd12-8549-4690-9e17-b858278507b3']

In [9]:
roll_up_multiplicity_for_type(lpg, lpg.nodes['165f4cd9-9849-45bf-b31b-d0b675df6774'], "upper")

2

In [10]:
from pymbe.interpretation.interp_playbooks import build_expression_sequence_templates
from pymbe.label import get_label_for_id

In [11]:
expr_seqs = build_expression_sequence_templates(lpg)
for seq in expr_seqs:
    sig_seq = []
    for item in seq:
        sig_seq.append(get_label_for_id(item, helper_client.elements_by_id))
    print(sig_seq)

['263cc821-b999-4f2c-8701-dab0f2e7da29 «Feature»', 'driveshaft.shaftPort_c ($collection) => $result', 'FRE.shaftPort_c', '$result: ShaftPort_c']
['263cc821-b999-4f2c-8701-dab0f2e7da29 «Feature»', 'driveshaft.shaftPort_c ($collection) => $result', 'FRE.driveshaft', '$result: Driveshaft']
['263cc821-b999-4f2c-8701-dab0f2e7da29 «Feature»', 'driveshaft.shaftPort_c ($collection) => $result', '$result']
['263cc821-b999-4f2c-8701-dab0f2e7da29 «Feature»', 'driveshaft.shaftPort_c ($collection) => $result', '$collection']
['95e4a736-2f48-4522-8a47-09b50039024a «Feature»', 'leftWheel.wheelToAxlePort ($collection) => $result', 'FRE.wheelToAxlePort', '$result: WheelToAxlePort']
['95e4a736-2f48-4522-8a47-09b50039024a «Feature»', 'leftWheel.wheelToAxlePort ($collection) => $result', '$collection']
['95e4a736-2f48-4522-8a47-09b50039024a «Feature»', 'leftWheel.wheelToAxlePort ($collection) => $result', '$result']
['95e4a736-2f48-4522-8a47-09b50039024a «Feature»', 'leftWheel.wheelToAxlePort ($collection

In [12]:
m0_interpretation = random_generator_playbook(
    helper_client,
    lpg,
    shorten_pre_bake
)

c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:284: UserWarning: These edge types are not in the graph: {'ResultExpressionMembership'}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")


DrivePwrPort, id = 46cef5c8-5d4c-459c-a3dd-c1144fdec0cf
VehicleToRoadPort, id = 6baf5adc-0c92-4688-81f9-275860c59c1b
Wheel, id = afc897ef-d6ba-4a15-a6be-b947f3e1016f
FuelCmdPort, id = 2b56fb10-ae39-4ad2-8553-4befc69fed38
HalfAxle, id = d4d665ab-73aa-482a-ad4f-3cb809baabcb
AxleToWheelPort, id = 02e4c404-5ee5-4a8c-8d11-4ac106ad4ff4
Engine, id = e22a9b97-a4a9-43e0-8bc9-f2dd108d6ff4
Differential, id = 2bfde858-7dec-451c-b11c-d2c67ac2433b
ShaftPort_b, id = 422de9ce-1d5a-45cf-b5a8-e72a7805d649
VehicleA, id = 1b474d84-19b7-41af-a3ce-346e36b3337b
WheelToRoadPort, id = b246e8d2-4209-4eb6-adf5-1289a9254ce9
ShaftPort_a, id = 5746b17d-4b0e-4226-a09b-6aa387bd52a6
ShaftPort_d, id = fa8514f6-3b63-459e-ae28-f5db8385364f
RearAxleAssembly, id = 9fb93a39-2fc6-4dcb-8188-1e8ba49cbf0d
ClutchPort, id = 2ed547be-eaf1-4ed4-ab0a-6da15e3b98df
WheelToAxlePort, id = 99487ee9-106c-4236-b320-0565d641eefb
AxlePort, id = 165f4cd9-9849-45bf-b31b-d0b675df6774
RearAxle, id = 15bc358c-a417-43ae-af4e-f9d5e0b1a143
Driveshaf

NotImplementedError: ('Cannot handle untyped features! Tried on 71bc413b-a68d-484a-94bc-0203da46eb71 «InterfaceUsage»', ' id = 71bc413b-a68d-484a-94bc-0203da46eb71')

To see how sequences are structured, the cell below renders sequences that show what type of atoms will fill particular positions in the sequence, as well as the maximum multiplicity (number of) sequences.

In [ ]:
from pymbe.query.query import roll_up_upper_multiplicity, roll_up_multiplicity_for_type

feat_sequences = build_sequence_templates(lpg=lpg)

total = 0
for seq in feat_sequences:
    print(str(pprint_single_id_list(seq, lpg.nodes)) + ", " + str(roll_up_upper_multiplicity(lpg, lpg.nodes[seq[-1]])))

Once the interpretations are generated, we can look for expressions and create an execution order (this is similar to Excel builds a dependency graph internally to accelerate computations and partial updates when a user changes a cell value).

In [ ]:
dcg = generate_execution_order(lpg, m0_interpretation)

One of the core tools in examining and working with the M1 model is using the get_projection function on the master graph to select out the kind of nodes and edges that will support other queries (roll-up using breadth-first search in reverse order, paths from one node to another to lay out sequences, etc.)

In [ ]:
from pymbe.graph.calc_lpg import CalculationGroup
cg = CalculationGroup(lpg.get_projection("Expression Inferred Graph"), m0_interpretation, dcg)

Display the calculation order determined by the algorithm, as well as a hint about what the step is (generating output from a function, moving values from function parameters to an attribute, applying redefinition, etc.)

In [ ]:
for item in dcg:
    instance_safe = True
    if item[0] not in m0_interpretation or len(m0_interpretation[item[0]]) == 0:
        print("No instances for " + lpg.nodes[item[0]]['qualifiedName'])
        instance_safe = False
    if item[1] not in m0_interpretation or len(m0_interpretation[item[1]]) == 0:
        print("No instances for " + lpg.nodes[item[1]]['qualifiedName'])
        instance_safe = False
    if instance_safe:
        rep_source = m0_interpretation[item[0]][0][-1]
        rep_target = m0_interpretation[item[1]][0][-1]
        if len(item) == 3:
            safe_item = item[2]
        else:
            safe_item = 'None'
        print('(' + str(rep_source) + ', ' + str(rep_target) + ', ' + safe_item + ')')

Use the calculation order in order to resolve the "unset" fields on many attributes to values where they are determined in the M1 model (e.g., using the ' = ' operator to assign values directly or connect to equations or analyses).

In [ ]:
cg.solve_graph(lpg)

## Calculation Results Shown

The following cells are a series of displays of relevant features in the interpretation.

In [ ]:
m0_interpretation[collect_1_result]

In [ ]:
m0_interpretation[collect_2_result]

In [ ]:
m0_interpretation[sum_2_collection]

In [ ]:
m0_interpretation[sum_1_result]

In [ ]:
m0_interpretation[x]

In [ ]:
m0_interpretation[liquid_stage_full_mass]

In [ ]:
m0_interpretation[fre_1_result]

In [ ]:
m0_interpretation[fre_2_result]

Show all interpretation sequence sets (limited to length of 5).

In [ ]:
for print_line in pprint_interpretation(m0_interpretation, lpg.nodes):
    print(print_line)